# Custom Layers

In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images = training_images / 255.0
test_images = test_images / 255.0

#### Classic model

In [34]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,)),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(10,activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])
model.fit(training_images,training_labels,epochs=5,validation_data=(test_images,test_labels))
model.evaluate(test_images, test_labels)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4997 - accuracy: 0.8250 - val_loss: 0.4413 - val_accuracy: 0.8431
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3761 - accuracy: 0.8633 - val_loss: 0.3931 - val_accuracy: 0.8612
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3366 - accuracy: 0.8769 - val_loss: 0.3702 - val_accuracy: 0.8674
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3109 - accuracy: 0.8867 - val_loss: 0.3631 - val_accuracy: 0.8684
Epoch 5/5
313/313 [==============================] - 0s 1ms/step - loss: 0.3568 - accuracy: 0.8708


[0.35676950216293335, 0.8708000183105469]

#### Lambda layer

With custom layer we still don't have the controll to set up the `activation` function, therefore we use `Lambda` function to workaround.

In [8]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,)),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Lambda(lambda x: tf.abs(x)),
    tf.keras.layers.Dense(10,activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])
model.fit(training_images,training_labels,epochs=5,validation_data=(test_images,test_labels))
model.evaluate(test_images, test_labels)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.5148 - accuracy: 0.8227 - val_loss: 0.4602 - val_accuracy: 0.8368
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3932 - accuracy: 0.8606 - val_loss: 0.4253 - val_accuracy: 0.8534
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3572 - accuracy: 0.8725 - val_loss: 0.4141 - val_accuracy: 0.8607
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3330 - accuracy: 0.8818 - val_loss: 0.3867 - val_accuracy: 0.8647
Epoch 5/5
313/313 [==============================] - 0s 1ms/step - loss: 0.4004 - accuracy: 0.8612


[0.4003850221633911, 0.8611999750137329]

---

In [14]:
def my_relu(x):
    return tf.keras.backend.maximum(-0.1, x)

In [15]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,)),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Lambda(my_relu),
    tf.keras.layers.Dense(10,activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])
model.fit(training_images,training_labels,epochs=5,validation_data=(test_images,test_labels))
model.evaluate(test_images, test_labels)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4895 - accuracy: 0.8270 - val_loss: 0.4318 - val_accuracy: 0.8488
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3716 - accuracy: 0.8659 - val_loss: 0.3923 - val_accuracy: 0.8557
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3327 - accuracy: 0.8781 - val_loss: 0.3793 - val_accuracy: 0.8620
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3105 - accuracy: 0.8855 - val_loss: 0.3491 - val_accuracy: 0.8746
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.3595 - accuracy: 0.8688


[0.3594619929790497, 0.8687999844551086]

#### Custom layer

With custom layer we still don't have the controll to set up the `activation` function, therefore we use `Lambda` function to workaround.

In [11]:
class CustomLayer(tf.keras.layers.Layer):
    def __init__(self,units=32):
        super(CustomLayer,self).__init__()
        self.units=units
    def build(self,input_shape):
        w_init=tf.random_normal_initializer()
        self.w=tf.Variable(name='kernel',initial_value=w_init(shape=(input_shape[-1],self.units),dtype='float32'),trainable=True)
        b_init=tf.zeros_initializer()
        self.b=tf.Variable(name='bias',initial_value=b_init(shape=(self.units,),dtype='float32'),trainable=True)
    def call(self,inputs):
        return tf.matmul(inputs,self.w)+ self.b

In [12]:
def my_relu(x):
    return tf.keras.backend.maximum(-0.1, x)

In [13]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,)),
    CustomLayer(128),
    tf.keras.layers.Lambda(my_relu),
    tf.keras.layers.Dense(10,activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])
model.fit(training_images,training_labels,epochs=5,validation_data=(test_images,test_labels))
model.evaluate(test_images, test_labels)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4979 - accuracy: 0.8243 - val_loss: 0.4185 - val_accuracy: 0.8542
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3705 - accuracy: 0.8662 - val_loss: 0.3796 - val_accuracy: 0.8665
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3358 - accuracy: 0.8778 - val_loss: 0.3632 - val_accuracy: 0.8684
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3111 - accuracy: 0.8862 - val_loss: 0.3858 - val_accuracy: 0.8633
Epoch 5/5
313/313 [==============================] - 0s 1ms/step - loss: 0.3988 - accuracy: 0.8595


[0.3988169729709625, 0.859499990940094]

In [14]:
layer=CustomLayer(1)

x=tf.ones((1,1))
y=layer(x)

layer.variables

[<tf.Variable 'custom_layer_5/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[0.00631469]], dtype=float32)>,
 <tf.Variable 'custom_layer_5/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

---

In [16]:
# xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0])
# ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0])

# layer=CustomLayer(1)
# model = tf.keras.models.Sequential([
#     layer
# ])

# model.compile(optimizer='sgd', loss='mean_squared_error')
# model.fit(xs, ys, epochs=500,verbose=0)

# print(model.predict([10.0]))
# print(layer.variables)

#### Custom Layer with activation

In [4]:
class CustomLayer(tf.keras.layers.Layer):
    def __init__(self,units=32,activation='linear'):
        super(CustomLayer,self).__init__()
        self.units=units
        self.activation=tf.keras.activations.get(activation)
    def build(self,input_shape):
        w_init=tf.random_normal_initializer()
        self.w=tf.Variable(name='kernel',initial_value=w_init(shape=(input_shape[-1],self.units),dtype='float32'),trainable=True)
        b_init=tf.zeros_initializer()
        self.b=tf.Variable(name='bias',initial_value=b_init(shape=(self.units,),dtype='float32'),trainable=True)
    def call(self,inputs):
        return self.activation(tf.matmul(inputs,self.w)+ self.b)

In [48]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,)),
    CustomLayer(128,activation='softmax'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10,activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])
model.fit(training_images,training_labels,epochs=5,validation_data=(test_images,test_labels))
model.evaluate(test_images, test_labels)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 1.4812 - accuracy: 0.5637 - val_loss: 0.9672 - val_accuracy: 0.7287
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 1.0156 - accuracy: 0.6215 - val_loss: 0.7266 - val_accuracy: 0.7452
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.9364 - accuracy: 0.6334 - val_loss: 0.6523 - val_accuracy: 0.7480
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.9068 - accuracy: 0.6339 - val_loss: 0.6162 - val_accuracy: 0.7573
Epoch 5/5
313/313 [==============================] - 0s 1ms/step - loss: 0.5993 - accuracy: 0.7728


[0.5993068814277649, 0.7728000283241272]